In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_vt.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #2269

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,919676,919676,3094947107,4/11/20,150,PPP,PC CONSTRUCTION COMPANY,193 TILLEY DR,SOUTH BURLINGTON,VT,...,Unanswered,NaN,9851647.79,6/11/21,23.0,Construction,VERMONT DISTRICT OFFICE,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491
1,919677,919677,5465908809,4/17/21,150,PPP,BRATTLEBORO RETREAT,1 Anna Marsh Lane,Brattleboro,VT,...,Unanswered,Y,7836548.19,9/16/21,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629
2,919678,919678,5465897006,4/5/20,150,PPP,GW PLASTICS INC,239 PLEASANT ST,BETHEL,VT,...,Unanswered,NaN,6043387.43,6/11/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580
3,919679,919679,3949557104,4/12/20,150,PPP,BENNINGTON COLLEGE CORPORATION,1 COLLEGE DR,BENNINGTON,VT,...,Unanswered,Y,6023558.50,8/10/21,61.0,Educational Services,VERMONT DISTRICT OFFICE,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727
4,919680,919680,5449437008,4/5/20,150,PPP,MOMENTUM MANFACTURING GROUP LLC,210 PIERCE RD,SAINT JOHNSBURY,VT,...,Unanswered,NaN,5654613.59,7/7/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2264,921940,921940,8117198404,2/12/21,150,PPS,STARK MOUNTAIN WOODWORKS CO.,359 South St,New Haven,VT,...,Unanswered,NaN,150900.00,9/24/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"359 South St, New Haven, VT",44.118951,-73.152154
2265,921941,921941,8325598705,4/7/21,150,PPS,NORTHERN HOSPITALITY LIMITED PARTNERSHIP,39 S Main St,White River Junction,VT,...,Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,VERMONT DISTRICT OFFICE,"39 S Main St, White River Junction, VT",43.648022,-72.318954
2266,921942,921942,8759258607,3/25/21,150,PPS,JULI A. LARSON P.C.,1100 Hinesburg Rd Ste 201,South Burlington,VT,...,Non-Veteran,NaN,NaN,NaN,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060
2267,921943,921943,8777268309,1/30/21,150,PPP,DEEP ANALYTICS LLC,56 College St Ste 201,Montpelier,VT,...,Unanswered,NaN,150752.05,8/4/21,54.0,"Professional, Scientific, and Technical Services",VERMONT DISTRICT OFFICE,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #1

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head()

,LoanNumber,full_add,Lat,Long
0,3094947107,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491
1,5465908809,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629
2,5465897006,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580
3,3949557104,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727
4,5449437008,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/VT_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [7]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #7204

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('50')] #VT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

2269


Matching FIPS to DataFrame

In [11]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:7204]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3094947107,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,[500070040024012]
1,5465908809,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,[500259686001003]
2,5465897006,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,[500279668003046]
3,3949557104,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,[500039710002045]
4,5449437008,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,[500059575001014]
...,...,...,...,...,...
2264,8117198404,"359 South St, New Haven, VT",44.118951,-73.152154,[500019604001123]
2265,8325598705,"39 S Main St, White River Junction, VT",43.648022,-72.318954,[500279656004029]
2266,8759258607,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060,[500070040024015]
2267,8777268309,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071,[500239548001015]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3094947107,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,500070040024012
1,5465908809,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,500259686001003
2,5465897006,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,500279668003046
3,3949557104,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,500039710002045
4,5449437008,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,500059575001014
...,...,...,...,...,...
2264,8117198404,"359 South St, New Haven, VT",44.118951,-73.152154,500019604001123
2265,8325598705,"39 S Main St, White River Junction, VT",43.648022,-72.318954,500279656004029
2266,8759258607,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060,500070040024015
2267,8777268309,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071,500239548001015


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #3

,LoanNumber,full_add,Lat,Long,FIPS_z
727,4395107002,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
793,4988897305,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2073,5687147010,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,3094947107,500070040024012
1,5465908809,500259686001003
2,5465897006,500279668003046
3,3949557104,500039710002045
4,5449437008,500059575001014
...,...,...
2264,8117198404,500019604001123
2265,8325598705,500279656004029
2266,8759258607,500070040024015
2267,8777268309,500239548001015


Read in base data

In [16]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_vt.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/WV_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 

#fips_check.head() #9179


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,919676,919676,3094947107,4/11/20,150,PPP,PC CONSTRUCTION COMPANY,193 TILLEY DR,SOUTH BURLINGTON,VT,...,Unanswered,NaN,9851647.79,6/11/21,23.0,Construction,VERMONT DISTRICT OFFICE,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491
1,919677,919677,5465908809,4/17/21,150,PPP,BRATTLEBORO RETREAT,1 Anna Marsh Lane,Brattleboro,VT,...,Unanswered,Y,7836548.19,9/16/21,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629
2,919678,919678,5465897006,4/5/20,150,PPP,GW PLASTICS INC,239 PLEASANT ST,BETHEL,VT,...,Unanswered,NaN,6043387.43,6/11/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580
3,919679,919679,3949557104,4/12/20,150,PPP,BENNINGTON COLLEGE CORPORATION,1 COLLEGE DR,BENNINGTON,VT,...,Unanswered,Y,6023558.50,8/10/21,61.0,Educational Services,VERMONT DISTRICT OFFICE,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727
4,919680,919680,5449437008,4/5/20,150,PPP,MOMENTUM MANFACTURING GROUP LLC,210 PIERCE RD,SAINT JOHNSBURY,VT,...,Unanswered,NaN,5654613.59,7/7/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2264,921940,921940,8117198404,2/12/21,150,PPS,STARK MOUNTAIN WOODWORKS CO.,359 South St,New Haven,VT,...,Unanswered,NaN,150900.00,9/24/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"359 South St, New Haven, VT",44.118951,-73.152154
2265,921941,921941,8325598705,4/7/21,150,PPS,NORTHERN HOSPITALITY LIMITED PARTNERSHIP,39 S Main St,White River Junction,VT,...,Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,VERMONT DISTRICT OFFICE,"39 S Main St, White River Junction, VT",43.648022,-72.318954
2266,921942,921942,8759258607,3/25/21,150,PPS,JULI A. LARSON P.C.,1100 Hinesburg Rd Ste 201,South Burlington,VT,...,Non-Veteran,NaN,NaN,NaN,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060
2267,921943,921943,8777268309,1/30/21,150,PPP,DEEP ANALYTICS LLC,56 College St Ste 201,Montpelier,VT,...,Unanswered,NaN,150752.05,8/4/21,54.0,"Professional, Scientific, and Technical Services",VERMONT DISTRICT OFFICE,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071


Merge with MainFrame

In [17]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,919676,919676,3094947107,4/11/20,150,PPP,PC CONSTRUCTION COMPANY,193 TILLEY DR,SOUTH BURLINGTON,VT,...,NaN,9851647.79,6/11/21,23.0,Construction,VERMONT DISTRICT OFFICE,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,500070040024012
1,919677,919677,5465908809,4/17/21,150,PPP,BRATTLEBORO RETREAT,1 Anna Marsh Lane,Brattleboro,VT,...,Y,7836548.19,9/16/21,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,500259686001003
2,919678,919678,5465897006,4/5/20,150,PPP,GW PLASTICS INC,239 PLEASANT ST,BETHEL,VT,...,NaN,6043387.43,6/11/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,500279668003046
3,919679,919679,3949557104,4/12/20,150,PPP,BENNINGTON COLLEGE CORPORATION,1 COLLEGE DR,BENNINGTON,VT,...,Y,6023558.50,8/10/21,61.0,Educational Services,VERMONT DISTRICT OFFICE,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,500039710002045
4,919680,919680,5449437008,4/5/20,150,PPP,MOMENTUM MANFACTURING GROUP LLC,210 PIERCE RD,SAINT JOHNSBURY,VT,...,NaN,5654613.59,7/7/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,500059575001014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2264,921940,921940,8117198404,2/12/21,150,PPS,STARK MOUNTAIN WOODWORKS CO.,359 South St,New Haven,VT,...,NaN,150900.00,9/24/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"359 South St, New Haven, VT",44.118951,-73.152154,500019604001123
2265,921941,921941,8325598705,4/7/21,150,PPS,NORTHERN HOSPITALITY LIMITED PARTNERSHIP,39 S Main St,White River Junction,VT,...,NaN,NaN,NaN,72.0,Accomodation and Food Services,VERMONT DISTRICT OFFICE,"39 S Main St, White River Junction, VT",43.648022,-72.318954,500279656004029
2266,921942,921942,8759258607,3/25/21,150,PPS,JULI A. LARSON P.C.,1100 Hinesburg Rd Ste 201,South Burlington,VT,...,NaN,NaN,NaN,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060,500070040024015
2267,921943,921943,8777268309,1/30/21,150,PPP,DEEP ANALYTICS LLC,56 College St Ste 201,Montpelier,VT,...,NaN,150752.05,8/4/21,54.0,"Professional, Scientific, and Technical Services",VERMONT DISTRICT OFFICE,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071,500239548001015


<b>Fill FIPS Columns - if FIPS file

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

In [ ]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #16 , notna #9163 / 9179

Convert to File

In [18]:
fips_VT = fips_merge
#fips_WV = fips_merge_drop

In [5]:
fips_VT.to_csv("data/state_data/geo/geo_fips/VT_fips_scraped.csv") 

Review & Compare

In [5]:
geo_VT_fips = pd.read_csv("data/state_data/geo/geo_fips/VT_fips_scraped.csv") 
geo_VT_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,919676,919676,3094947107,4/11/20,150,PPP,...,N,9851647.79,6/11/21,23.0,Construction,VERMONT DISTRICT OFFICE,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14
1,1,1,1,1,919677,919677,5465908809,4/17/21,150,PPP,...,Y,7836548.19,9/16/21,62.0,Health Care and Social Assistance,VERMONT DISTRICT OFFICE,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14
2,2,2,2,2,919678,919678,5465897006,4/5/20,150,PPP,...,N,6043387.43,6/11/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14
3,3,3,3,3,919679,919679,3949557104,4/12/20,150,PPP,...,Y,6023558.50,8/10/21,61.0,Educational Services,VERMONT DISTRICT OFFICE,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14
4,4,4,4,4,919680,919680,5449437008,4/5/20,150,PPP,...,N,5654613.59,7/7/21,33.0,Manufacturing,VERMONT DISTRICT OFFICE,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14


In [3]:
##### fill in missing Lat, Long, FIPS
geo_VT_fips.loc[geo_VT_fips["FIPS_z"].isnull()] #9

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
727,727,727,727,920403,920403,4395107002,4/3/20,150,PPP,DERBY CHRYSLER DODGE JEEP INC.,...,N,426799.61,3/31/21,44.0,Retail Trade,VERMONT DISTRICT OFFICE,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
793,793,793,793,920469,920469,4988897305,4/30/20,150,PPP,"CONNOR CONTRACTING, INC.",...,N,393993.91,3/5/21,23.0,Construction,VERMONT DISTRICT OFFICE,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2073,2073,2073,2073,921749,921749,5687147010,4/6/20,150,PPP,"ANDREW PEARCE BOWLS, LLC",...,N,164353.17,11/12/20,42.0,Wholesale Trade,VERMONT DISTRICT OFFICE,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN
